# Look-Aside Cache for MongoDB
### This is a sample notebook for using Aerospike as a read/look-aside cache

- This notebook demonstrates the use of Aerospike as a cache using Mongo as another primary datastore
- It is required to run Mongo as a separte container using `docker run --name some-mongo -d mongo:latest`

To test: Run the `cache.getData("id", "data");` method once - to fetch from Mongo and populate Aerospike

Another run will fetch the data from Aerospike cache


#### Ensure that Aerospike Database is running

In [2]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

#### Load Aerospike and Mongo dependencies from POM

In [6]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
  <dependency>
    <groupId>org.mongodb</groupId>
    <artifactId>mongo-java-driver</artifactId>
    <version>3.12.7</version>
  </dependency>
</dependencies>

In [7]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.mongodb.client.MongoDatabase;
import com.mongodb.client.MongoCollection;
import com.mongodb.MongoClient; 
import com.mongodb.MongoCredential; 
import org.bson.Document;
import com.mongodb.client.model.Filters;
import java.util.Set;

## Configure the clients

The configuration is for 
 - Aerospike database running on port 3000 of localhost (IP 127.0.0.1) which is the default. 
 - Mongo running in a separate container whose IP can be found by `docker inspect <containerid> | grep -i ipaddress`


Modify config if your environment is different (Aerospike database running on a different host or different port).

In [8]:
public class Cache{
        //Database Constants
        public static final String AEROSPIKE_HOST = "0.0.0.0";
        public static final String MONGO_HOST = "172.17.0.3";
        public static final int AEROSPIKE_PORT = 3000;
        public static final int MONGO_PORT = 27017;
        
        public static final String AEROSPIKE_NAMESPACE = "test";
        public static final String AEROSPIKE_SET = "demo";
        public static final String MONGO_USER = "sampleUser";
        public static final String MONGO_PASSWORD = "password";
        public static final String MONGO_DB = "myDb";
        public static final String MONGO_COLLECTION = "sampleCollection";
        
        private AerospikeClient client;
        private MongoClient mongo;
        private MongoCredential credential;
        private MongoDatabase database;
        
        public Cache() {
            client = new AerospikeClient(AEROSPIKE_HOST, AEROSPIKE_PORT);
            mongo = new MongoClient(MONGO_HOST , MONGO_PORT);
            credential = MongoCredential.createCredential(MONGO_USER, MONGO_DB, 
                                 MONGO_PASSWORD.toCharArray());
            database = mongo.getDatabase(MONGO_DB);
        }
        
        private boolean collectionExists(final String collectionName) {
            // Check and return if the collection exists in Mongo
            return database.listCollectionNames()
                .into(new ArrayList<String>()).contains(collectionName);
        }

        public void populateMongoData(String id, String data) {
            // Populate Mongodb first
            Document document = new Document(id, data);
            if (! collectionExists(MONGO_COLLECTION)) {
                database.createCollection(MONGO_COLLECTION);
            } else {
                MongoCollection<Document> collection = database.getCollection(MONGO_COLLECTION);
                collection.insertOne(document);
            }
            Key key = new Key(AEROSPIKE_NAMESPACE, AEROSPIKE_SET, id);
            client.delete(null, key);
        }
        
        public String getData(String id, String data) {
            // This is just an example code that exhibits a cache fetch for a String id with String data
            
            Key key = new Key(AEROSPIKE_NAMESPACE, AEROSPIKE_SET, id);
            String BIN_NAME = "value";
            Record record = client.get(null,key);
            if ( record == null ) {
                System.out.println("First Fetch Record does not exist in Aerospike cache");
                MongoCollection<Document> collection = database.getCollection(MONGO_COLLECTION);
                Document document = collection.find(Filters.eq(id, data)).first();
                //System.out.println("Document " + document.get(id));
                String json = document.get(id).toString();
                client.put(null, key, new Bin(BIN_NAME,json));
                return client.get(null, key).toString();  
            
            } else {
                System.out.println("Data retrieved from Aerospike cache");
                return record.toString();
                
            }
        }
}

In [9]:
Cache cache = new Cache();
cache.populateMongoData("id", "data");

In [10]:
cache.getData("id", "data");


First Fetch Record does not exist in Aerospike cache


(gen:1),(exp:350708590),(bins:(value:data))